In [1]:
import sqlite3
import re
import json
import pandas as pd
import time
import numpy as np
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

In [2]:
def create_sqlite_connector(db='UltimateGuitarTabs.db'):
    con = sqlite3.connect('../data/{}'.format(db))
    cur = con.cursor()
    return(con, cur)
        
def get_table(tableName):
    cur.execute("SELECT * FROM "+tableName)
    
    return(cur.fetchall())

def exe_query(query):
    cur.execute(query)

    return(cur.fetchall())

In [3]:
def fix_accidental(note, accidental):
    notes = np.asarray(['A', 'B', 'C', 'D', 'E', 'F', 'G'])

    note_idx = int(np.where(notes == note)[0])
    if accidental == '#':
        # Account for B# or E#
        if note == 'B':
            return('C', '')
        elif note == 'E':
            return('F', '')
        return(note,accidental)
    elif accidental == 'b':
        # Account for Cb or Fb
        if note == 'C':
            return('B', '')
        elif note == 'F':
            return('E', '')
        return(notes[note_idx - 1],'#')
    else:
        return(note, accidental)
            
        
def clean_chords(chords):
    """
    This function takes in a comma-separated string of 
    chords and cleans it by removing any base note variations, or
    other chord embelishments.  Diminished labels are kept as these
    are used in the chord progression table. The purpose of this
    is to clean the chords to match the labels within the chord
    progression table.
    
    returns:
        new_chords - array of newly cleaned chords to be tabulated
                        by the chord progression table
    """
    
    # Pattern grouping: 1=(chord pitch) 2=(base note) 3=(chord type) 4=(base note)
    pattern = "^([A-G]+)(\/[A-G]*[b#])*([(?m)|(?m\d)|(?b\d)|(?#\d)|(?maj\d)|\
    (?add\d)|(?sus\d)|(?aug)|(?aug\d)|(?dim)|(?dim\d)]*)(\/[A-G]*[b#])*"        
    prog = re.compile(pattern)

    pattern2 = "^([A-G])([b#])?(m$|m\d$)?(dim$|dim\d$)?"
    prog2 = re.compile(pattern2)

    chords = chords.split(',')

    new_chords = [""]*len(chords)
    for i in range(len(chords)):
        curr_chord = chords[i]
        groups = prog.findall(curr_chord)[0] 
        no_base = groups[0] + groups[2]
        no_num = re.sub(pattern="\d", repl="", string=no_base)

        groups = prog2.findall(no_num)[0]
        note,accidental = fix_accidental(groups[0], groups[1])
        new_chords[i] = note + accidental + groups[2] + groups[3]

    return(new_chords)


def get_key_tbls():
    """
    Helper function that reads in a chord progression table
    and creates a dictionary that maps chord names to their
    indices on the chord progression table. This dictionary
    will be used to tabulate a 'key table' to determine the
    key of a song.
    
    returns:
        Key_dict - Dictionary mapping chord names to indices
        Keys - array of keys that correspond to the order
                of the progression table
    """
    Key_tbl = pd.read_csv('../data/key_table.csv')
    Keys = list(Key_tbl.key)
    Tbl = np.asmatrix(Key_tbl.iloc[:,1:8])

    # Storing all possible chords 
    all_chords = []
    for i in range(Tbl.shape[0]):
        for j in np.asarray(Tbl[i])[0]:
            all_chords.append(j)
    all_chords = np.unique(all_chords)
    
    # Creating dict(key='chord', val='indices in progression tbl')
    Key_dict = {}
    for chord in all_chords:
        Key_dict[chord] = np.where(Tbl == chord)

    return(Key_dict, Keys)

def is_rel_min(comp_key, act_key):
    """
    Determines if the actual key marked indicated
    UltimateGuitarTabs.com is the relative minor
    of the actual key (i.e., Am(rel. minor) and C(actual))
    
    args:
        comp_key - Computed key
        act_key - UltimateGuitarTabs.com key
    returns:
        - True if it's relative minor, False otherwise
    """
    notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
    if str(act_key[-1]) != 'm': #If not minor, return 
        return(False)
    if len(comp_key) == 2:
        note, accidental = fix_accidental(comp_key[0],comp_key[1])
        comp_key = note + accidental
    comp_idx = np.where(np.asarray(notes) == comp_key)[0][0]
    rel_min_idx = comp_idx - 3
    if notes[rel_min_idx] + 'm' == act_key:
        return(True)

def compute_key(chords):
    """
    Computes the key of a song by analyzing
    the chords used within the song. A theoretical
    key matrix where each row specifies the types of chords
    that coincide with a key is used to tabulate
    the existing chords. The largest row sum is the
    computed key of the song. 
    
    args:
        Key_dict - Python dictionary containing all possible
                   chords and their indexes in the theoretical key matrix.
                   This is used to tabulate a matrix of zeros.
        Keys - List of possible keys corresponding to theoretical key matrix
        chords - String of chords separated by commas
                   
    returns:
        - The computed key
        - List of cleaned chords, only major or minor
    """

    Key_dict, Keys = get_key_tbls() # Grabbing key dictionary and keys

    chords = clean_chords(chords)
    count_mat = np.zeros((12,7)) # Matrix of zeros to tabulate chord occurences
    # Tabulating chords
    for chord in chords:
        count_mat[Key_dict[chord]] += 1
        
    computed_key = Keys[np.argmax(np.sum(count_mat, axis = 1))]
    return(computed_key, chords)

def transpose_C(chords, states, song_id):
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    comp_key, chords = compute_key(chords)
    new_chords = [""]*len(chords) # Empty list for new chords
    if comp_key == 'C':
        for chord in chords:
            states.add(chord)
        return(states, \
               {'song_id' : song_id, \
                'orig_key': comp_key, \
                'trans_chords': chords})
    else:
        steps = np.where(np.asarray(notes) == comp_key)[0][0]
        for i in range(len(chords)):
            is_min = False
            is_dim = False
            curr_chord = chords[i]
            if curr_chord[-1] == 'm':
                if curr_chord[-3:len(curr_chord)] == 'dim':
                    is_dim = True
                    curr_chord = curr_chord[0:-3]
                else:
                    is_min = True
                    curr_chord = curr_chord[0:-1]
            
            chord_idx = np.where(np.asarray(notes) == curr_chord)[0][0]
            new_chord_idx = chord_idx - steps
            if is_min:
                new_chords[i] = notes[new_chord_idx] + 'm'
            elif is_dim:
                new_chords[i] = notes[new_chord_idx] + 'dim'
            else:
                new_chords[i] = notes[new_chord_idx]
            states.add(new_chords[i])
        return(states, \
               {'song_id' : song_id, \
                'orig_key': comp_key, \
                'trans_chords': new_chords})

def get_clean_chord_progressions():
    """
    This function will rewrite the Chords table transposing all progressions
    to the key of C. While doing so, it also keeps track of all the different
    chords exist in the "states" variable.
    """
    songs = [""]*Chords.shape[0]
    states = set()
    for i in range(Chords.shape[0]):
        states, songs[i] = transpose_C(Chords['Chords'][i], states, Chords['Id'][i])
    states = sorted(list(states))
    return(states, songs)

def write_clean_songs(songs):
    delete = "DROP TABLE IF EXISTS Clean_Chords;"
    cur.execute(delete)
    
    create = "CREATE TABLE Clean_Chords (id integer primary key, orig_key text, chords text)"
    cur.execute(create)
    
    for i in range(len(songs)):
        
        song = songs[i]
        song_id = int(song['song_id'])
        orig_key = song['orig_key']
        chords = ','.join(song['trans_chords'])
        
        sql_tab = "INSERT INTO Clean_Chords (id,orig_key,chords) \
        VALUES ('%d','%s','%s')" % \
        (song_id, orig_key, chords) 
        
        try:
            cur.execute(sql_tab)
            con.commit()
        except:
            print('Unable to insert into Clean_chords')

def write_states(states):
    delete = "DROP TABLE IF EXISTS States"
    cur.execute(delete)
    
    create_states = "CREATE TABLE States (state text)"
    cur.execute(create_states)
    
    sql_tab = "INSERT INTO States (state) VALUES (?)"
    try:
        cur.executemany(sql_tab, [(s,) for s in states])
        con.commit()
    except:
        print('Unable to insert into States')
            

In [4]:
con, cur = create_sqlite_connector()
main_query = """
    SELECT Chords.id, Song, Artist, Tonality, Capo, Chords, num_hits, votes, rating 
    FROM Chords INNER JOIN Hits ON Chords.id = Hits.id
    """
Chords = pd.DataFrame(list(exe_query(main_query)),
                      columns=['Id', 'Song', 'Artist', 'Key', 'Capo', 'Chords', 'Num_hits', 'Votes', 'Rating'])

In [5]:
Chords.head()

,Id,Song,Artist,Key,Capo,Chords,Num_hits,Votes,Rating
0,23,Beer Drinkers And Hell Raisers,ZZ Top,,0,"Em7,E,D,A5,A6,B5,B6,Em7,Em7,E,D,A5,A6,A5,A6,Em...",1405271,26,4.18630
1,73,Harvest Moon,Neil Young,,0,"G,C,C,G,C,G,C,C,G,C,G,C,C,G,C,G,C,G,G,Bm,C,Am,...",1519389,42,4.65345
2,74,Unplugged,Neil Young,,0,"G,C,G,G,C,G,C,G,C,G,D,C,G,G,C,G,G,C,G,C,C,Am,C...",1384175,22,4.61619
3,105,Ball And Chain,XTC,,0,"C,C7sus4,C,C7sus4,C,C7sus4,Bb,F,C,G5,D,F,C,G5,...",1402850,0,0.00000
4,118,Los Angeles,X,,0,"E,C,D,G,A,A,G,D,C,A,G,D,C,A,G,C,A,G,C,A,G,C,A,...",1393927,18,4.49543


In [6]:
states, cleaned_chords = get_clean_chord_progressions()

In [7]:
write_clean_songs(cleaned_chords)
write_states(states)

In [8]:
Cleaned_Chords = pd.DataFrame(list(get_table('Clean_Chords')), 
                              columns = ['Id', 'orig_key', 'trans_chords'])
Cleaned_Chords.head()

,Id,orig_key,trans_chords
0,23,E,"Cm,C,A#,F,F,G,G,Cm,Cm,C,A#,F,F,F,F,Cm,G,G,G,G,..."
1,73,G,"C,F,F,C,F,C,F,F,C,F,C,F,F,C,F,C,F,C,C,Em,F,Dm,..."
2,74,G,"C,F,C,C,F,C,F,C,F,C,G,F,C,C,F,C,C,F,C,F,F,Dm,F..."
3,105,C,"C,C,C,C,C,C,A#,F,C,G,D,F,C,G,D,F,C,G,D,F,C,G,D..."
4,118,G,"A,F,G,C,D,D,C,G,F,D,C,G,F,D,C,F,D,C,F,D,C,F,D,..."


In [9]:
States = pd.DataFrame(list(get_table('States')), 
                      columns = ['States'])
States

,States
0,A
1,A#
2,A#dim
3,A#m
4,Adim
5,Am
6,B
7,Bdim
8,Bm
9,C
